## Version 0.2 <br>
__Changelog__ <br>
1. Added ability to input sequence and have it map each particle to its correct type. <br>
2. Harmonic bonds can now be created automatically.<br>
3. Making a letter lowecase in the sequence allows you to make it a rigid type. <br>
4. Particle 164 no longer moves faster than the speed of light.<br>
5. The simulation now runs (tested it to 10e7 steps with no errors).<br>
6. Can now add patches to rigid bodies. <br>
<br> 

__To Do__ <br>

1. Assign appropriate bond lengths. <br>
2. Figure out particle radii (hydrated radius? molecule length? The paper "The Packing Density in Proteins: Standard Radii and Volumes" outlines a Voronoi approach to generate radii.). <br>
3. Figure out/ assign bond angles and appropriate forces. <br>
4. Figure out/ assign appropriate LJ and Yukawa potentials. <br>
5. Maybe give special tag to C-Terminal codon? Paper used it to determine density. <br>
6. Revise classes as needed. <br>
7. Develop system for automatically creating an optimal box size.<br>
9. Develop system to automatically calculate moments of inertia by checking neighbor masses. <br>
10. etc. <br>

## Sequence. Bold = patchy regions: <br>

MR 

AMQ __ALMQIQQ GLQTLATEAP GLIPSFTPGV GVGVLGTAIG PVGPVTPI__ GP

I __GPIVPFT__ PI GPIGPIGPTG PAAPPGSTGS GGPTGPTVSS AAPSETTSPT

SESGPN __QQFI QQMVQALAGA NA__ PQLPNPEV RFQQQLEQLN __ANGFL__ NREAN

LQALIATGGD INAAIE __RLLG SQ__ PSW
<br><br>
AMQ __almqiqq glqtlateap glipsftpgv gvgvlgtaig pvgpvtpi__ GP

I __gpivpft__ PI GPIGPIGPTG PAAPPGSTGS GGPTGPTVSS AAPSETTSPT

SESGPN __qqfi qqmvqalaga na__ PQLPNPEV RFQQQLEQLN __angfl__ NREAN

LQALIATGGD INAAIE __rllg sq__ PSW

## Full sequence (https://www.uniprot.org/uniprot/Q9UHD9)

        10         20         30         40         50
MAENGESSGP PRPSRGPAAA QGSAAAPAEP KIIKVTVKTP KEKEEFAVPE 

        60         70         80         90        100
NSSVQQFKEA ISKRFKSQTD QLVLIFAGKI LKDQDTLIQH GIHDGLTVHL 

       110        120        130        140        150
VIKSQNRPQG QSTQPSNAAG TNTTSASTPR SNSTPISTNS NPFGLGSLGG 

       160        170        180        190        200
LAGLSSLGLS STNFSELQSQ MQQQLMASPE MMIQIMENPF VQSMLSNPDL 

       210        220        230        240        250
MRQLIMANPQ MQQLIQRNPE ISHLLNNPDI MRQTLEIARN PAMMQEMMRN 

       260        270        280        290        300
QDLALSNLES IPGGYNALRR MYTDIQEPML NAAQEQFGGN PFASVGSSSS 

       310        320        330        340        350
SGEGTQPSRT ENRDPLPNPW APPPATQSSA TTSTTTSTGS GSGNSSSNAT 

       360        370        380        390        400
GNTVAAANYV ASIFSTPGMQ SLLQQITENP QLIQNMLSAP YMRSMMQSLS 

       410        420        430        440        450
QNPDLAAQMM LNSPLFTANP QLQEQMRPQL PAFLQQMQNP DTLSAMSNPR 

       460        470        480        490        500
AMQALMQIQQ GLQTLATEAP GLIPSFTPGV GVGVLGTAIG PVGPVTPIGP 

       510        520        530        540        550
IGPIVPFTPI GPIGPIGPTG PAAPPGSTGS GGPTGPTVSS AAPSETTSPT 

       560        570        580        590        600
SESGPNQQFI QQMVQALAGA NAPQLPNPEV RFQQQLEQLN AMGFLNREAN 

       610        620 
LQALIATGGD INAAIERLLG SQPS  

__Notes__: <br>

All but types W and D have patches associated with them.<br>

- __Hydrophobic:__ A, G, I, L, M, P, V <br> 
- __Aromatic:__ F, W, Y <br> 
- __Basic:__ H, K, R <br>
- __Acidic:__ D, E <br>
- __Polar:__ C, N, Q, S, T <br>

## Sources <br>
1. C=N bond length https://www.ncbi.nlm.nih.gov/books/NBK22364/
2. more bond lengths and angles https://mcl1.ncifcrf.gov/dauter_pubs/171.pdf
3. more bond lengths (see paper I got)
4. Sequence determinants of protein phase behavior from a coarse-grained model.

- C-N 1.327 Å 
- C=N 1.27 Å
- C-O 1.237 Å
- Cα-C 1.508 Å
- Cα-Cβ 1.512 Å
- etc.

## Testing each section of code as I write it. This section of code is incomplete and is being worked on. The next cell has the completed patchy polymer code as a reference. 

## Just make rigid centers lowercase and modify seq_list to control each one individually. can replace entire sections of the sequence with letters if we go more general. 

In [2]:
import hoomd,imp
import hoomd
from hoomd import *
from hoomd import md
import numpy as np
import gsd.hoomd
import time 
import matplotlib.pyplot as plt
from decimal import *
from inspect import currentframe
import string

class SimBox:
    
    ###Creates Initial Snapshot
    def MakeSnapshot(self, boxsize, nparticles): 
        
        snapshot=hoomd.data.make_snapshot(N=nparticles,box=hoomd.data.boxdim(L=boxsize+nparticles),
                                          bond_types=['tether'],particle_types=self.GenParticleTypes(nparticles, debug=True))
        return snapshot
    
    
    #Only used for initializing the system, do not use outside of MakeSnapshot!
    def GenParticleTypes(self, nparticles, debug=None):
        counter=0
        #List that will contain the particle types. Enable debug to print.
        GPT = [] 
        for k in range(nparticles):
            counter=k
            GPT.append('C%d'%counter)
        if(debug==True):
            print("DEBUG: GenParticleTypes: Particle Types = ",GPT)
        return GPT
class Polymer:

    def SetPolyProperties(self, nparticles, boxsize, diam=None, debug=None):
        if(diam==None):
            diam=1.0 #default diameter
        for i in range(nparticles):
            system.particles.get(i).position = [-(boxsize-nparticles)+(((boxsize-nparticles))/nparticles)*i,0,0]
            system.particles.get(i).diameter = 1.0
            #system.particles.get(i).type = ('C%d'%(i))
            if(debug==True):
                print("DEBUG: SetPolyProperties: Particle %d is at "%i, [nparticles+i,0,0])
    
    #MakePolyTage is just a debugging tool, ignore. 
    def MakePolyTags(self, nparticles, debug=None):
        P_Tag_List = []
        for i in range(nparticles):
            system.particles.get(i).type=('C%d'%(i+1))
            P_Tag_List.append('C%d'%(i+1))
        if(debug==True):
            print("DEBUG: SetPolyProperties: P_Tag_List is ", P_Tag_List)
        
    def PatchTypes(self, tot_npatch, debug=None):
        patch_types = []
        for i in range(tot_npatch):
            system.particles.types.add('%d'%(i+1))
            patch_types.append('%d'%(i+1))
        if(debug==True):
            print("DEBUG: PatchTypes: ",patch_types)
        return patch_types

            
    
    def MakeBonds(self, nparticles, debug=None):
        DebugBonds = []
        f=0
        for f in range(nparticles-1):
            system.bonds.add('tether',f,f+1)
            DebugBonds.append([f,f+1])
            f+=1
        if(debug==True):
            print("DEBUG: MakeBonds: Bonds = ", DebugBonds)  
class Debug:
    def PrintAllParticleTags(self, nparticles, tot_npatch):
        X = []
        for i in range(nparticles+tot_npatch):
            X.append(system.particles[i].tag)
        print("DEBUG: List of All Particle Tags: ", X)
    def PrintSysParticles(self, debug=None):
        if(debug==True):
            for i in range(len(system.particles)):
                print(system.particles[i])
    def PrintLJPairs(self, sig, types):
        for i in range(0,len(sig)):
            for j in range(i,len(sig)):
                print([types[i],types[j]])
    def DebugMessage(self, string, variable, debug=None):
        if(debug==True):
            cf = currentframe()
            print("%s"%string, variable, " at line ", cf.f_back.f_lineno)
    def DebugInfo(self):
        print("Debug Message Format Is As Follows:\n DEBUG: Function: Parameter = Value at line <linenumber>")

class MomentOfInertia:
  
    #Principle Moment Calculator
    #added an option to return the inertia tensor for debugging
    def Moment(self, coord, Mass, return_tensor = None, debug=None):
        DB = Debug()
        #Referencing local variables
        cx,cy,cz = 0.0,0.0,0.0
        x,y,z = 0.0,0.0,0.0
        cXYZ = []
        Ixx,Ixy,Ixz,Iyy,Iyz,Izz = 0.0,0.0,0.0,0.0,0.0,0.0
        TotalMass = sum(Mass)
        if(debug==True):
            print("DEBUG: Moment: len(coord)= ", len(coord))
            print("DEBUG: Moment: TotalMass = ", TotalMass)
        for i in range(len(Mass)):
                x += Mass[i]*coord[i][0]
                y += Mass[i]*coord[i][1]
                z += Mass[i]*coord[i][2]
                if(debug==True):
                    print("DEBUG: Moment: x = Mass[%d]"%i +"*coord[%d][0]"%i +" =%f "%x)
                    print("DEBUG: Moment: y = Mass[%d]"%i +"*coord[%d][1]"%i +" =%f "%y)
                    print("DEBUG: Moment: z = Mass[%d]"%i +"*coord[%d][2]"%i +" =%f "%z)
        cx = x/TotalMass
        cy = y/TotalMass
        cz = z/TotalMass
        if(debug==True):
            print("DEBUG: Moment: cx = x/TotalMass = ", cx)
            print("DEBUG: Moment: cy = y/TotalMass = ", cy)
            print("DEBUG: Moment: cz = y/TotalMass = ", cz)
        com = [cx,cy,cz]
        for i in range(len(coord)):
            cXYZ.append([coord[i][0]-com[0], coord[i][1]-com[1], coord[i][2]-com[2]]) 
        if(debug==True):
            print("DEBUG: Moment: cXYZ = ", cXYZ)
            print("DEBUG: Moment: len(cXYZ) = ", len(cXYZ))
        #Constructs Inertia Tensor
        #Useful Property: Ixy=Iyx, Ixz=Izx, Iyz=Izy
        for i in range(len(coord)):
            Ixx += Mass[i]*(cXYZ[i][1]**2 + cXYZ[i][2]**2)
            Iyy += Mass[i]*(cXYZ[i][0]**2 + cXYZ[i][2]**2)
            Izz += Mass[i]*(cXYZ[i][0]**2 + cXYZ[i][1]**2)
            Ixy += -Mass[i]*cXYZ[i][0]*cXYZ[i][1]
            Ixz += -Mass[i]*cXYZ[i][0]*cXYZ[i][2]
            Iyz += -Mass[i]*cXYZ[i][1]*cXYZ[i][2]
        if(debug==True):
            print("DEBUG: Moment: Ixx = ", Ixx)
            print("DEBUG: Moment: Iyy = ", Iyy)
            print("DEBUG: Moment: Izz = ", Izz)
            print("DEBUG: Moment: Ixy = ", Ixy)
            print("DEBUG: Moment: Ixz = ", Ixz)
            print("DEBUG: Moment: Iyz = ", Iyz)

        #Computes eignevalues, finds principle moments
        Imatrix = np.matrix([[Ixx,Ixy,Ixz],[Ixy,Iyy,Iyz],[Ixz,Iyz,Izz]])
        Idiag = np.linalg.eig(Imatrix)
        if(debug==True):
            print("DEBUG: Moment: Idiag = ",Idiag)
            print("DEBUG: Moment: Idiag[0] = ",Idiag[0])
        comX = 0-com[0]
        comY = 0-com[1] 
        for i in range(len(coord)):
            coord[i] = [coord[i][0]+comX,coord[i][1]+comY,coord[i][2]]
        if(return_tensor==True):
            return Idiag[0], Imatrix
        else:
            return Idiag[0]
#Just for flexability 
def Angle(angle):
    angle = angle*np.pi/180
    return angle
DB = Debug()
DB.DebugInfo()
SB = SimBox()
MoI = MomentOfInertia()
Poly = Polymer()
#Filename (date and time will be appended onto the file name)
filename = "UBQLN2"
#Box Size
boxsize = 5
#Number of constituent particles
nparticles = 177 #will need to be 1
#Total number of patches in sim
#tot_npatch = 79
tot_npatch = 28
#Rigid Diameter
diam = 1.0
#List of Patch Diameters
pdiam = [1.0]
#LJ Cutoff
rcut = 2.5
#Random Seed
SEED = np.random.randint(0,100000000)
#Empty list for setting up sig values
Sigma = []
#Empty list for setting up eps values
Epsilon = []
#Empty list for yukawa kappa
YukKap = []
#Empty list for yukawa eps
YukEps = []
#Allows me to make unique file names
t = time.strftime(time.strftime("%d %b %H:%M:%S", time.gmtime()))
#Harmonic Bond Length
hbl = 3.8
##----------SETUP----------##
context.initialize("")
amino_masses = dict([['A',89.094],['C',121.150],['D',133.103],['E',147.130],['F',165.192],['G',75.067],
                    ['H',155.157],['I',131.175],['K',146.19],['L',131.175],['M',149.210],['N',132.119],
                    ['P',115.132],['Q',146.146],['R',174.204],['S',105.093],['T',119.120],['V',117.148],
                    ['W',204.229],['Y',181.191],
                    ['a',89.094],['c',121.150],['d',133.103],['e',147.130],['f',165.192],['g',75.067],
                    ['h',155.157],['i',131.175],['k',146.19],['l',131.175],['m',149.210],['n',132.119],
                    ['p',115.132],['q',146.146],['r',174.204],['s',105.093],['t',119.120],['v',117.148],
                    ['w',204.229],['y',181.191]])

#From source [4]
amino_diam = dict([['A',5.04],['C',5.48],['D',5.58],['E',5.92],['F',6.36],['G',4.50],
                    ['H',6.08],['I',6.18],['K',6.36],['L',6.18],['M',6.18],['N',5.68],
                    ['P',5.56],['Q',6.02],['R',6.56],['S',5.18],['T',5.62],['V',5.86],
                    ['W',6.78],['Y',6.46],['a',5.04],['c',5.48],['d',5.58],['e',5.92],['f',6.36],['g',4.50],
                    ['h',6.08],['i',6.18],['k',6.36],['l',6.18],['m',6.18],['n',5.68],
                    ['p',5.56],['q',6.02],['r',6.56],['s',5.18],['t',5.62],['v',5.86],
                    ['w',6.78],['y',6.46]])
#Complete Sequence
#sequence = 'MRAMQALMQIQQGLQTLATEAPGLIPSFTPGVGVGVLGTAIGPVGPVTPIGPIGPIVPFTPIGPIGPIGPTGpAAPPGSTGSGGPTGPTVSSAAPSETTSPTSESGPNQQFIQQMVQALAGANAPQLPnPEVRFQQQLEQLNANGFLNREANLQALIATGGDINAAIERLLGSQPSW'     
sequence = 'MRAMQalmqiqqglqtlateapglipsftpgvgvgvlgtaigpvgpvtpiGPIgpivpftPIGPIGPIGPTGPAAPPGSTGSGGPTGPTVSSAAPSETTSPTSESGPNqqfiqqmvqalaganaPQLPNPEVRFQQQLEQLNangflNREANLQALIATGGDINAAIErllgsqPSW'      
seq_list = []
rigid_tags = []
for i in range(len(sequence)):
    seq_list.append('%s'%sequence[i])
for i in range(len(sequence)):
    for j in range(26):
        if(sequence[i] is string.ascii_lowercase[j]):
            rigid_tags.append(i)
def unique(list1): #from https://www.geeksforgeeks.org/python-get-unique-values-list/
    # intilize a null list 
    unique_list = [] 
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    return unique_list
#Creates a list of all amino acid types in polymer
seq_types = unique(seq_list)



#Creates snapshot, initializes particle types
snapshot=hoomd.data.make_snapshot(N=nparticles,box=hoomd.data.boxdim(Lx=3000, Ly=100, Lz=100),
        bond_types=['tether'],particle_types=seq_types)

system = hoomd.init.read_snapshot(snapshot)

#Sets each particle's type equal to it's amino acid
for i in range(nparticles):
    system.particles.get(i).type = ('%s'%seq_list[i])
    system.particles.get(i).diameter = amino_diam['%s'%seq_list[i]]
    if(i<nparticles-1):
        system.particles.get(i).position = [-(750)+i*hbl+amino_diam['%s'%seq_list[i]]+amino_diam['%s'%seq_list[i+1]],0,0]
    else:
        system.particles.get(i).position = [-(750)+i*hbl+amino_diam['%s'%seq_list[i]]+amino_diam['%s'%seq_list[i-1]],0,0]
    

# for i in range(nparticles):
#     system.particles.get(i).type = ('%s'%seq_list[i])
#     system.particles.get(i).diameter = amino_diam['%s'%seq_list[i]]
# #     if(i<nparticles-1):
# #         print(i)

# #         system.particles.get(i).position = [(i*hbl)+amino_diam['%s'%seq_list[i]] + amino_diam['%s'%seq_list[i+1]],0,0]
# #     elif(i>=nparticles):
 
# #         system.particles.get(i).position = [(i*hbl)+amino_diam['%s'%seq_list[i]]+amino_diam['%s'%seq_list[i-1]],0,0]
    


# ix = 0
# placeholder = 0
# for p in system.particles:
#     if(ix<nparticles-1):
#         print(ix)
#         placeholder = -500 + (ix*hbl*3)+amino_diam['%s'%seq_list[ix]] + amino_diam['%s'%seq_list[ix+1]]
        
#         p.position = [-500 + (ix*hbl*3)+amino_diam['%s'%seq_list[ix]] + amino_diam['%s'%seq_list[ix+1]],0,0]
#         print((ix*hbl*3)+amino_diam['%s'%seq_list[ix]] + amino_diam['%s'%seq_list[ix+1]])
#         ix +=1
#     elif(ix>=nparticles):
#         p.position = [placeholder + (ix*hbl+3.8)+amino_diam['%s'%seq_list[ix]]+amino_diam['%s'%seq_list[ix-1]],0,0]
#         ix+=1
    
##Adds patch Patch Types from [1,tot_npatch]
##----------Assign Masses----------##
for p in system.particles:
    p.mass = amino_masses[p.type]
    
patch_types = Poly.PatchTypes(tot_npatch, debug=False) 
types = system.particles.types
DB.DebugMessage("DEBUG: types = ", types, debug=True)


##----------Builds Dictionaries for LJ Parameters---------##
for i in range(len(types)):
    if(i>=(len(types)-tot_npatch)):
        Sigma.append(('%s'%types[i], 1.0))
        Epsilon.append(('%s'%types[i], 0.5)) 
    elif(i<len(types)):
        Sigma.append(('%s'%types[i], 1.0))
        Epsilon.append(('%s'%types[i], 0.1))  
sig = dict(Sigma)
DB.DebugMessage("DEBUG: sig = ", sig, debug = True)
eps = dict(Epsilon)
DB.DebugMessage("DEBUG: eps = ", eps, debug = True)


##----------Builds Dictionaries for Yukawa Parameters---------##
    ##Allows independent assigning of parameters of patches and particles
for i in range(len(types)):
    if(i>=(len(types)-tot_npatch)): #i=npaticles is where the indices for the patch tags start
        YukKap.append(('%s'%types[i], 1.0))
        YukEps.append(('%s'%types[i], 2.0)) 
    elif(i<len(types)):
        YukKap.append(('%s'%types[i], 0.0))
        YukEps.append(('%s'%types[i], 0.0)) 
yukkap = dict(YukKap)
DB.DebugMessage("DEBUG: yukkap = ", yukkap, debug = True)
yukeps = dict(YukEps)
DB.DebugMessage("DEBUG: yukeps = ", yukeps, debug = True)

###----------Make Rigid Bodies----------###
rigid=hoomd.md.constrain.rigid()
coord_patch_group_a = [[2.52*np.cos(Angle(60)),0,2.52*np.sin(Angle(60))],
                       [2.52*np.cos(Angle(60)),0,-2.52*np.sin(Angle(60))]]

coord_patch_group_l = [[3.09*np.cos(Angle(60)),0,3.09*np.sin(Angle(60))],
                       [3.09*np.cos(Angle(60)),0,-3.09*np.sin(Angle(60))]]

coord_patch_group_m = [[3.09*np.cos(Angle(60)),0,3.09*np.sin(Angle(60))],
                       [3.09*np.cos(Angle(60)),0,-3.09*np.sin(Angle(60))]]

coord_patch_group_q = [[3.01*np.cos(Angle(60)),0,3.01*np.sin(Angle(60))],
                       [3.01*np.cos(Angle(60)),0,-3.01*np.sin(Angle(60))]]

coord_patch_group_i = [[3.09*np.cos(Angle(60)),0,3.09*np.sin(Angle(60))],
                       [3.09*np.cos(Angle(60)),0,-3.09*np.sin(Angle(60))]]

coord_patch_group_g = [[2.25*np.cos(Angle(60)),0,2.25*np.sin(Angle(60))],
                       [2.25*np.cos(Angle(60)),0,-2.25*np.sin(Angle(60))]]

coord_patch_group_t = [[2.81*np.cos(Angle(60)),0,2.81*np.sin(Angle(60))],
                       [2.81*np.cos(Angle(60)),0,-2.81*np.sin(Angle(60))]]

coord_patch_group_e = [[2.96*np.cos(Angle(60)),0,2.96*np.sin(Angle(60))],
                       [2.96*np.cos(Angle(60)),0,-2.96*np.sin(Angle(60))]]

coord_patch_group_p = [[2.78*np.cos(Angle(60)),0,2.78*np.sin(Angle(60))],
                       [2.78*np.cos(Angle(60)),0,-2.78*np.sin(Angle(60))]]

coord_patch_group_s = [[2.59*np.cos(Angle(60)),0,2.59*np.sin(Angle(60))],
                       [2.59*np.cos(Angle(60)),0,-2.59*np.sin(Angle(60))]]

coord_patch_group_f = [[3.18*np.cos(Angle(60)),0,3.18*np.sin(Angle(60))],
                       [3.18*np.cos(Angle(60)),0,-3.18*np.sin(Angle(60))]]

coord_patch_group_v = [[2.93*np.cos(Angle(60)),0,2.93*np.sin(Angle(60))],
                       [2.93*np.cos(Angle(60)),0,-2.93*np.sin(Angle(60))]]

coord_patch_group_n = [[2.84*np.cos(Angle(60)),0,2.84*np.sin(Angle(60))],
                       [2.84*np.cos(Angle(60)),0,-2.84*np.sin(Angle(60))]]

coord_patch_group_r = [[3.28*np.cos(Angle(60)),0,3.28*np.sin(Angle(60))],
                       [3.28*np.cos(Angle(60)),0,-3.28*np.sin(Angle(60))]]

rigid_neigh_1 =[[0.5*np.cos(Angle(60)),0,0.5*np.sin(Angle(60))],
                [0.5*np.cos(Angle(60)),0,-0.5*np.sin(Angle(60))],
                [-hbl,0,0],[hbl,0,0]]

mass_patch_group_1 = [0.5,0.5,75.067,89.094]
mass_patch_group_2 = [0.5,0.5,115.132,115.132]
# types_patch_group_1 = ['1','2']
diameters_patch = [0.25,0.25]
eigen_group_1 = MoI.Moment(rigid_neigh_1, mass_patch_group_1, debug=False)
eigen_group_2 = MoI.Moment(rigid_neigh_1, mass_patch_group_2, debug=False)
#['a', 'l', 'm', 'q', 'i', 'g', 't', 'e', 'p', 's', 'f', 'v', 'n', 'r']

rigid.set_param('a',positions=coord_patch_group_a, types=['1','2'], diameters = diameters_patch)
rigid.set_param('l',positions=coord_patch_group_l, types=['3','4'], diameters = diameters_patch)
rigid.set_param('m',positions=coord_patch_group_m, types=['5','6'], diameters = diameters_patch)
rigid.set_param('q',positions=coord_patch_group_q, types=['7','8'], diameters = diameters_patch)
rigid.set_param('i',positions=coord_patch_group_i, types=['9','10'], diameters = diameters_patch)
rigid.set_param('g',positions=coord_patch_group_g, types=['11','12'], diameters = diameters_patch)
rigid.set_param('t',positions=coord_patch_group_t, types=['13','14'], diameters = diameters_patch)
rigid.set_param('e',positions=coord_patch_group_e, types=['15','16'], diameters = diameters_patch)
rigid.set_param('p',positions=coord_patch_group_p, types=['17','18'], diameters = diameters_patch)
rigid.set_param('s',positions=coord_patch_group_s, types=['19','20'], diameters = diameters_patch)
rigid.set_param('f',positions=coord_patch_group_f, types=['21','22'], diameters = diameters_patch)
rigid.set_param('v',positions=coord_patch_group_v, types=['23','24'], diameters = diameters_patch)
rigid.set_param('n',positions=coord_patch_group_n, types=['25','26'], diameters = diameters_patch)
rigid.set_param('r',positions=coord_patch_group_r, types=['27','28'], diameters = diameters_patch)

rigid.create_bodies()

center = hoomd.group.rigid_center()
nonrigid = hoomd.group.nonrigid()
part = hoomd.group.all()
gpoly = hoomd.group.union(name='gpoly', a=center, b=nonrigid) 
patches = hoomd.group.difference(name = 'patches', a=part, b=gpoly)
for p in patches:
    p.mass = 0.5
for i in range(len(rigid_tags)):
    Z = rigid_tags[i]
    system.particles[Z].moment_inertia = eigen_group_1

##----------Make Bonds----------##
bi = 0 #bond index
DebugBonds = []
for bi in range(nparticles-1):
    system.bonds.add('tether',bi,bi+1)
    DebugBonds.append([bi,bi+1])
    bi+=1
harm = md.bond.harmonic()
harm.bond_coeff.set('tether',k=20,r0=hbl)


###----------Make Angles----------###
###Work in progress, make dict of angles. Look at particle types to deternine the right angle to use. 
    ###Get types: type[i-3],type[i-2],type[i-1]
    ### 
# h=3
# while(h<=(nparticles-1)):
#     system.angles.add("angleA", h-3, h-2, h-1)
#     h+=1
# harmonic = hoomd.md.angle.harmonic()
# harmonic.angle_coeff.set('angleA', k=50.0, t0=Angle(120))

nl = md.nlist.cell(r_buff = 0.4, check_period = 1)

yukawa = md.pair.yukawa(r_cut=3.0, nlist=nl)
lj = md.pair.lj(r_cut=rcut, nlist=nl)


##----------Generates LJ pairs----------##
for i in range(0,len(sig)):
    for j in range(i,len(sig)):
        lj.pair_coeff.set(types[i],types[j],
                epsilon=0.5*(eps[types[i]]+eps[types[j]]),
                sigma=0.5*(sig[types[i]]+sig[types[j]]))

##----------Generates Yukawa pairs----------##
for i in range(0,len(yukkap)):
    for j in range(i,len(yukkap)):
        yukawa.pair_coeff.set(types[i],types[j],
                epsilon=(yukeps[types[i]]+yukeps[types[j]]),
                kappa=(yukkap[types[i]]+yukkap[types[j]])) 
hoomd.md.integrate.mode_standard(dt=0.002)
#hoomd.md.integrate.langevin(group=hoomd.group.all(), kT=0.1, seed = 5)
hoomd.md.integrate.langevin(group=nonrigid, kT=0.1, seed = SEED)
hoomd.md.integrate.langevin(group=center, kT=0.1, seed = SEED)
hoomd.dump.gsd(filename='{0} {1}.gsd'.format(filename, t),period=1000, group = hoomd.group.all() ,overwrite = True, dynamic=['attribute','property','momentum','topology'])
hoomd.run(1e4)


Debug Message Format Is As Follows:
 DEBUG: Function: Parameter = Value at line <linenumber>
notice(2): Group "all" created containing 177 particles
DEBUG: types =  Particle types: ['M', 'R', 'A', 'Q', 'a', 'l', 'm', 'q', 'i', 'g', 't', 'e', 'p', 's', 'f', 'v', 'G', 'P', 'I', 'T', 'S', 'V', 'E', 'N', 'n', 'L', 'F', 'D', 'r', 'W', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28']  at line  295
DEBUG: sig =  {'M': 1.0, 'R': 1.0, 'A': 1.0, 'Q': 1.0, 'a': 1.0, 'l': 1.0, 'm': 1.0, 'q': 1.0, 'i': 1.0, 'g': 1.0, 't': 1.0, 'e': 1.0, 'p': 1.0, 's': 1.0, 'f': 1.0, 'v': 1.0, 'G': 1.0, 'P': 1.0, 'I': 1.0, 'T': 1.0, 'S': 1.0, 'V': 1.0, 'E': 1.0, 'N': 1.0, 'n': 1.0, 'L': 1.0, 'F': 1.0, 'D': 1.0, 'r': 1.0, 'W': 1.0, '1': 1.0, '2': 1.0, '3': 1.0, '4': 1.0, '5': 1.0, '6': 1.0, '7': 1.0, '8': 1.0, '9': 1.0, '10': 1.0, '11': 1.0, '12': 1.0, '13': 1.0, '14': 1.0, '15': 1.0, '16': 1.0, '17': 1.0, '1

*Warning*: Integrator #0: Anisotropic integration requested, but no rotational degrees of freedom found for its group


In [13]:
print(pindex)

59980.75999999999


In [1]:
import string

sequence = 'AMQalmqiqqglqtlateapglipsftpgvgvgvlgtaigpvgpvtpiGPIgpivpftPIGPIGPIGPTGPAAPPGSTGSGGPTGPTVSSAAPSETTSPTSESGPNqqfiqqmvqalaganaPQLPNPEVRFQQQLEQLNangflNREANLQALIATGGDINAAIErllgsqPSW'      
seq_list = []
rigid_tags = []
r_types = []
for i in range(len(sequence)):
    seq_list.append('%s'%sequence[i])
for i in range(len(sequence)):
    for j in range(26):
        if(sequence[i] is string.ascii_lowercase[j]):
            rigid_tags.append(i)
            r_types.append(sequence[i])
def unique(list1): #from https://www.geeksforgeeks.org/python-get-unique-values-list/
    # intilize a null list 
    unique_list = [] 
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    return unique_list
#Creates a list of all amino acid types in polymer
seq_types = unique(seq_list)
rigid_types = unique(r_types)
print("ST = ",seq_types)
print("RT = ", rigid_types)




ST =  ['A', 'M', 'Q', 'a', 'l', 'm', 'q', 'i', 'g', 't', 'e', 'p', 's', 'f', 'v', 'G', 'P', 'I', 'T', 'S', 'V', 'E', 'N', 'n', 'L', 'R', 'F', 'D', 'r', 'W']
RT =  ['a', 'l', 'm', 'q', 'i', 'g', 't', 'e', 'p', 's', 'f', 'v', 'n', 'r']


In [14]:
print(DebugBonds)

[[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 8], [8, 9], [9, 10], [10, 11], [11, 12], [12, 13], [13, 14], [14, 15], [15, 16], [16, 17], [17, 18], [18, 19], [19, 20], [20, 21], [21, 22], [22, 23], [23, 24], [24, 25], [25, 26], [26, 27], [27, 28], [28, 29], [29, 30], [30, 31], [31, 32], [32, 33], [33, 34], [34, 35], [35, 36], [36, 37], [37, 38], [38, 39], [39, 40], [40, 41], [41, 42], [42, 43], [43, 44], [44, 45], [45, 46], [46, 47], [47, 48], [48, 49], [49, 50], [50, 51], [51, 52], [52, 53], [53, 54], [54, 55], [55, 56], [56, 57], [57, 58], [58, 59], [59, 60], [60, 61], [61, 62], [62, 63], [63, 64], [64, 65], [65, 66], [66, 67], [67, 68], [68, 69], [69, 70], [70, 71], [71, 72], [72, 73], [73, 74], [74, 75], [75, 76], [76, 77], [77, 78], [78, 79], [79, 80], [80, 81], [81, 82], [82, 83], [83, 84], [84, 85], [85, 86], [86, 87], [87, 88], [88, 89], [89, 90], [90, 91], [91, 92], [92, 93], [93, 94], [94, 95], [95, 96], [96, 97], [97, 98], [98, 99], [99, 100], [100, 101], [101,

In [27]:
string.ascii_lowercase

'abcdefghijklmnopqrstuvwxyz'

In [30]:
print(rigid_tags)

[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 51, 52, 53, 54, 55, 56, 57, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 140, 141, 142, 143, 144, 166, 167, 168, 169, 170, 171]


## (for reference, main code in above cell)

In [ ]:
# import hoomd,imp
# import hoomd
# from hoomd import *
# from hoomd import md
# import numpy as np
# import gsd.hoomd
# import time 
# import matplotlib.pyplot as plt
# from decimal import * #for data analysis later
# from inspect import currentframe

# class SimBox:
    
#     ###Creates Initial Snapshot
#     def MakeSnapshot(self, boxsize, nparticles): 
        
#         snapshot=hoomd.data.make_snapshot(N=nparticles,box=hoomd.data.boxdim(L=boxsize+nparticles),
#                                           bond_types=['tether'],particle_types=self.GenParticleTypes(nparticles, debug=True))
#         return snapshot
    
    
#     #Only used for initializing the system, do not use outside of MakeSnapshot!
#     def GenParticleTypes(self, nparticles, debug=None):
#         counter=0
#         #List that will contain the particle types. Enable debug to print.
#         GPT = [] 
#         for k in range(nparticles):
#             counter=k
#             GPT.append('C%d'%counter)
#         if(debug==True):
#             print("DEBUG: GenParticleTypes: Particle Types = ",GPT)
#         return GPT
# class Polymer:

#     def SetPolyProperties(self, nparticles, boxsize, diam=None, debug=None):
#         if(diam==None):
#             diam=1.0 #default diameter
#         for i in range(nparticles):
#             system.particles.get(i).position = [-(boxsize-nparticles)+(((boxsize-nparticles))/nparticles)*i,0,0]
#             system.particles.get(i).diameter = 1.0
#             #system.particles.get(i).type = ('C%d'%(i))
#             if(debug==True):
#                 print("DEBUG: SetPolyProperties: Particle %d is at "%i, [nparticles+i,0,0])
    
#     #MakePolyTage is just a debugging tool, ignore. 
#     def MakePolyTags(self, nparticles, debug=None):
#         P_Tag_List = []
#         for i in range(nparticles):
#             system.particles.get(i).type=('C%d'%(i+1))
#             P_Tag_List.append('C%d'%(i+1))
#         if(debug==True):
#             print("DEBUG: SetPolyProperties: P_Tag_List is ", P_Tag_List)
        
#     def PatchTypes(self, tot_npatch, debug=None):
#         patch_types = []
#         for i in range(tot_npatch):
#             system.particles.types.add('%d'%(i+1))
#             patch_types.append('%d'%(i+1))
#         if(debug==True):
#             print("DEBUG: PatchTypes: ",patch_types)
#         return patch_types

            
    
#     def MakeBonds(self, nparticles, debug=None):
#         DebugBonds = []
#         f=0
#         for f in range(nparticles-1):
#             system.bonds.add('tether',f,f+1)
#             DebugBonds.append([f,f+1])
#             f+=1
#         if(debug==True):
#             print("DEBUG: MakeBonds: Bonds = ", DebugBonds)  
# class Debug:
#     def PrintAllParticleTags(self, nparticles, tot_npatch):
#         X = []
#         for i in range(nparticles+tot_npatch):
#             X.append(system.particles[i].tag)
#         print("DEBUG: List of All Particle Tags: ", X)
#     def PrintSysParticles(self, debug=None):
#         if(debug==True):
#             for i in range(len(system.particles)):
#                 print(system.particles[i])
#     def PrintLJPairs(self, sig, types):
#         for i in range(0,len(sig)):
#             for j in range(i,len(sig)):
#                 print([types[i],types[j]])
#     def DebugMessage(self, string, variable, debug=None):
#         if(debug==True):
#             cf = currentframe()
#             print("%s"%string, variable, " at line ", cf.f_back.f_lineno)
#     def DebugInfo(self):
#         print("Debug Message Format Is As Follows:\n DEBUG: Function: Parameter = Value at line <linenumber>")

# class MomentOfInertia:
  
#     #Principle Moment Calculator
#     #added an option to return the inertia tensor for debugging
#     def Moment(self, coord, Mass, return_tensor = None, debug=None):
#         DB = Debug()
#         #Referencing local variables
#         cx,cy,cz = 0.0,0.0,0.0
#         x,y,z = 0.0,0.0,0.0
#         cXYZ = []
#         Ixx,Ixy,Ixz,Iyy,Iyz,Izz = 0.0,0.0,0.0,0.0,0.0,0.0
#         TotalMass = sum(Mass)
#         if(debug==True):
#             print("DEBUG: Moment: len(coord)= ", len(coord))
#             print("DEBUG: Moment: TotalMass = ", TotalMass)
#         for i in range(len(Mass)):
#                 x += Mass[i]*coord[i][0]
#                 y += Mass[i]*coord[i][1]
#                 z += Mass[i]*coord[i][2]
#                 if(debug==True):
#                     print("DEBUG: Moment: x = Mass[%d]"%i +"*coord[%d][0]"%i +" =%f "%x)
#                     print("DEBUG: Moment: y = Mass[%d]"%i +"*coord[%d][1]"%i +" =%f "%y)
#                     print("DEBUG: Moment: z = Mass[%d]"%i +"*coord[%d][2]"%i +" =%f "%z)
#         cx = x/TotalMass
#         cy = y/TotalMass
#         cz = z/TotalMass
#         if(debug==True):
#             print("DEBUG: Moment: cx = x/TotalMass = ", cx)
#             print("DEBUG: Moment: cy = y/TotalMass = ", cy)
#             print("DEBUG: Moment: cz = y/TotalMass = ", cz)
#         com = [cx,cy,cz]
#         for i in range(len(coord)):
#             cXYZ.append([coord[i][0]-com[0], coord[i][1]-com[1], coord[i][2]-com[2]]) 
#         if(debug==True):
#             print("DEBUG: Moment: cXYZ = ", cXYZ)
#             print("DEBUG: Moment: len(cXYZ) = ", len(cXYZ))
#         #Constructs Inertia Tensor
#         #Useful Property: Ixy=Iyx, Ixz=Izx, Iyz=Izy
#         for i in range(len(coord)):
#             Ixx += Mass[i]*(cXYZ[i][1]**2 + cXYZ[i][2]**2)
#             Iyy += Mass[i]*(cXYZ[i][0]**2 + cXYZ[i][2]**2)
#             Izz += Mass[i]*(cXYZ[i][0]**2 + cXYZ[i][1]**2)
#             Ixy += -Mass[i]*cXYZ[i][0]*cXYZ[i][1]
#             Ixz += -Mass[i]*cXYZ[i][0]*cXYZ[i][2]
#             Iyz += -Mass[i]*cXYZ[i][1]*cXYZ[i][2]
#         if(debug==True):
#             print("DEBUG: Moment: Ixx = ", Ixx)
#             print("DEBUG: Moment: Iyy = ", Iyy)
#             print("DEBUG: Moment: Izz = ", Izz)
#             print("DEBUG: Moment: Ixy = ", Ixy)
#             print("DEBUG: Moment: Ixz = ", Ixz)
#             print("DEBUG: Moment: Iyz = ", Iyz)

#         #Computes eignevalues, finds principle moments
#         Imatrix = np.matrix([[Ixx,Ixy,Ixz],[Ixy,Iyy,Iyz],[Ixz,Iyz,Izz]])
#         Idiag = np.linalg.eig(Imatrix)
#         if(debug==True):
#             print("DEBUG: Moment: Idiag = ",Idiag)
#             print("DEBUG: Moment: Idiag[0] = ",Idiag[0])
#         comX = 0-com[0]
#         comY = 0-com[1] 
#         for i in range(len(coord)):
#             coord[i] = [coord[i][0]+comX,coord[i][1]+comY,coord[i][2]]
#         if(return_tensor==True):
#             return Idiag[0], Imatrix
#         else:
#             return Idiag[0]
# #Filename (date and time will be appended onto the file name)
# filename = "UBQLN2"
# #Box Size
# boxsize = 5
# #Number of constituent particles
# nparticles = 177 #will need to be 1
# #Total number of patches in sim
# tot_npatch = 4
# #Rigid Diameter
# diam = 1.0
# #List of Patch Diameters
# pdiam = [1.0]
# #LJ Cutoff
# rcut = 2.5
# #Random Seed
# SEED = np.random.randint(0,100000000)
# #Empty list for setting up sig values
# Sigma = []
# #Empty list for setting up eps values
# Epsilon = []
# #Empty list for yukawa kappa
# YukKap = []
# #Empty list for yukawa eps
# YukEps = []
# #Harmonic Bond Length
# hbl = 1.46
# #Allows me to make unique file names
# t = time.strftime(time.strftime("%d %b %H:%M:%S", time.gmtime()))
# #Just for flexability 
# def Angle(angle):
#     angle = angle*np.pi/180
#     return angle
# ##----------SETUP----------##
# context.initialize("")
# amino_masses = dict(['A',89.094],['C',121.150],['D',133.103],['E',147.130],['F',165.192],['G',75.067],
#                     ['H',155.157],['I',131.175],['K',146.19],['L',131.175],['M',149.210],['N',132.119],
#                     ['P',115.132],['Q',146.146],['R',174.204],['S',105.093],['T',119.120],['V',117.148],
#                     ['W',204.229],['Y',181.191],
#                     ['a',89.094],['c',121.150],['d',133.103],['e',147.130],['f',165.192],['g',75.067],
#                     ['h',155.157],['i',131.175],['k',146.19],['l',131.175],['m',149.210],['n',132.119],
#                     ['p',115.132],['q',146.146],['r',174.204],['s',105.093],['t',119.120],['v',117.148],
#                     ['w',204.229],['y',181.191])
# # rigid_masses = dict(['a',89.094],['c',121.150],['d',133.103],['e',147.130],['f',165.192],['g',75.067],
# #                     ['h',155.157],['i',131.175],['k',146.19],['l',131.175],['m',149.210],['n',132.119],
# #                     ['p',115.132],['q',146.146],['r',174.204],['s',105.093],['t',119.120],['v',117.148],
# #                     ['w',204.229],['y',181.191])
# sequence = 'MRAMQALMQIQQGLQTLATEAPGLIPSFTPGVGVGVLGTAIGPVGPVTPIGPIGPIVPFTPIGPIGPIGPTGPAAPPGSTGSGGPTGPTVSSAAPSETTSPTSESgPNQQFIQQMVQALAGANAPQLPNPEVRFQQQLEQLNANGFLNREANLQaLIATGGDINAAIERLLGSQPSW'     
# seq_list = []
# for i in range(len(sequence)):
#     seq_list.append[sequence[i]]
# def unique(list1): #from https://www.geeksforgeeks.org/python-get-unique-values-list/
#     # intilize a null list 
#     unique_list = [] 
#     # traverse for all elements 
#     for x in list1: 
#         # check if exists in unique_list or not 
#         if x not in unique_list: 
#             unique_list.append(x) 
#     return unique_list
# seq_types = unique(seq_list)
# DB = Debug()
# DB.DebugInfo()
# SB = SimBox()
# MoI = MomentOfInertia()
# Poly = Polymer()

# #Creates snapshot, initializes particle types
# snapshot=hoomd.data.make_snapshot(N=nparticles,box=hoomd.data.boxdim(Lx=200, Ly=4, Lz=4),
#         bond_types=['tether'],particle_types=particle_types=SB.GenParticleTypes(nparticles, debug=False))

# system = hoomd.init.read_snapshot(snapshot)

# Poly.SetPolyProperties(nparticles, boxsize, diam=1.0, debug=False)

# #Assign placeholder types (C0,C1,...) to actual types according to the sequence
# index = 0
# for p in particles:
#     p.type = sequence[index]
#     system.particles.get(i).position = [-(200)+i*hbl,0,0]
#     index+=1
# DB.DebugMessage("DEBUG: types = ", types, debug=False)
# patch_types = Poly.PatchTypes(tot_npatch, debug=False) ##Adds patch Patch Types from [1,tot_npatch]
# types = system.particles.types
# ##----------Builds Dictionaries for LJ Parameters---------##
# for i in range(len(types)):
#     if(i>=nparticles):
#         Sigma.append(('%s'%types[i], 1.0))
#         Epsilon.append(('%s'%types[i], 0.5)) 
#     elif(i<nparticles):
#         Sigma.append(('%s'%types[i], 1.0))
#         Epsilon.append(('%s'%types[i], 0.1))  
# sig = dict(Sigma)
# eps = dict(Epsilon)
# ##----------Builds Dictionaries for Yukawa Parameters---------##
#     ##Allows independent assigning of parameters of patches and particles
# for i in range(len(types)):
#     if(i>=nparticles): #i=npaticles is where the indices for the patch tags start
#         YukKap.append(('%s'%types[i], 0.5))
#         YukEps.append(('%s'%types[i], 2.0)) 
#     elif(i<nparticles):
#         YukKap.append(('%s'%types[i], 0.0))
#         YukEps.append(('%s'%types[i], 0.0)) 
# yukkap = dict(YukKap)
# yukeps = dict(YukEps)

# rigid=hoomd.md.constrain.rigid()

# ##----------Patch Group 1----------##
# coord_patch_group_1 = [[np.cos(Angle(60)),0,np.sin(Angle(60))],
#                        [np.cos(Angle(60)),0,-np.sin(Angle(60))]]
# mass_patch_group_1 = [0.5,0.5]
# types_patch_group_1 = ['1','2']
# diameters_patch_group_1 = [1.0,1.0]

# ##----------Patch Group 2----------##
# coord_patch_group_2 = [[np.cos(Angle(60)),0,np.sin(Angle(60))],
#                        [np.cos(Angle(60)),0,-np.sin(Angle(60))]]
# mass_patch_group_2 = [0.5,0.5]
# types_patch_group_2 = ['3','4']
# diameters_patch_group_2 = [1.0,1.0]

# ##----------Calculates Moments of Inertia----------##
# eigen_group_1 = MoI.Moment(coord_patch_group_1, mass_patch_group_1, debug=False)
# eigen_group_2 = MoI.Moment(coord_patch_group_2, mass_patch_group_2, debug=False)

# ##----------Sets Rigid Parameters----------##
# rigid.set_param('C0',positions=coord_patch_group_1, types=['1','2'], diameters = [1.0,1.0])
# rigid.set_param('C2',positions=coord_patch_group_2, types=['3','4'], diameters = [1.0,1.0])

# rigid.create_bodies()

# ##----------Creates groups----------##
#     ##Grouping rigid centers after replication breaks everything.##
#     ##Extra groups for debugging and/or flexability##
# center = hoomd.group.rigid_center()
# nonrigid = hoomd.group.nonrigid()
# part = hoomd.group.all()
# gpoly = hoomd.group.union(name='gpoly', a=center, b=nonrigid) 
# patches = hoomd.group.difference(name = 'patches', a=part, b=gpoly) 

# for p in gpoly:
#     p.mass = 12.0
# for p in patches:
#     p.mass = 1.0
# for p in center:
#     p.moment_inertia = [0,0,0]

# system.particles[0].moment_inertia = eigen_group_1
# system.particles[2].moment_inertia = eigen_group_2

# Poly.MakeBonds(nparticles, debug=True)

# system.replicate(nx=5,ny=5,nz=5)

# harm = md.bond.harmonic()
# harm.bond_coeff.set('tether',k=20,r0=0.83)


# nl = md.nlist.cell(r_buff = 0.4, check_period = 1)
# #nl.tune(warmup=5e4, r_min=0.05, r_max=1.0, jumps=20, steps=5e4, r_buff)
# #nl.tune(warmup=200000, r_min=0.05, r_max=1.0, jumps=20, steps=5000, set_max_check_period=False, quiet=False)
# lj = md.pair.lj(r_cut=rcut, nlist=nl)
# tmp1,tmp2 = 0.0,0.0

# #Generates LJ pairs
# for i in range(0,len(sig)):
#     for j in range(i,len(sig)):
#         lj.pair_coeff.set(types[i],types[j],
#                 epsilon=0.5*(eps[types[i]]+eps[types[j]]),
#                 sigma=0.5*(sig[types[i]]+sig[types[j]]))

# #Generates Yukawa pairs
# for i in range(0,len(yukkap)):
#     for j in range(i,len(yukkap)):
#         yukawa.pair_coeff.set(types[i],types[j],
#                 epsilon=(yukeps[types[i]]+yukeps[types[j]]),
#                 kappa=(yukkap[types[i]]+yukkap[types[j]]))       
# #Integrators
# hoomd.md.integrate.mode_standard(dt=0.002)
# #hoomd.md.integrate.langevin(group=part, kT=0.1, seed = 5)
# hoomd.md.integrate.langevin(group=nonrigid, kT=0.01, seed = SEED)
# hoomd.md.integrate.langevin(group=center, kT=0.01, seed = SEED)



# ### DUMP AND RUN ###
# hoomd.analyze.log(filename='LOG {0}.log'.format(t),quantities=['time','num_particles','ndof','translational_ndof','rotational_ndof','potential_energy','kinetic_energy','translational_kinetic_energy',
# 'rotational_kinetic_energy','temperature','pressure','pair_lj_energy','pair_yukawa_energy','bond_harmonic_energy','angle_harmonic_energy],period=1000,header_prefix='#'
#                                                       ,overwrite=True)
# #hoomd.deprecated.dump.xml(group=part,filename ='random.xml',vis=True,image=True)
# hoomd.dump.gsd(filename='{0} {1}.gsd'.format(filename, t),period=1000, group = hoomd.group.all() ,overwrite = True, dynamic=['attribute'])
# hoomd.run(1e5)

In [ ]:
sequence = 'MRAMQALMQIQQGLQTLATEAPGLIPSFTPGVGVGVLGTAIGPVGPVTPIGPIGPIVPFTPIGPIGPIGPTGPAAPPGSTGSGGPTGPTVSSAAPSETTSPTSESGPNQQFIQQMVQALAGANAPQLPNPEVRFQQQLEQLNANGFLNREANLQALIATGGDINAAIERLLGSQPSW'     


In [ ]:
sequence[0]

In [ ]:
print(seq_list)

In [ ]:
system.particles.get(0).type = 'M'

In [ ]:
print(types)

In [3]:
for i in range(nparticles):
    print(system.particles[i])

tag         : 0
position    : (-86.99682549433354, 0.0035670170416863315, -0.008730772114329346)
image       : (0, 0, 0)
velocity    : (-0.0016986277705030724, 0.005376750767708713, -0.0022518545181883626)
acceleration: (0.10464797694924596, 0.1088470094752283, 0.04603598458684272)
charge      : 0.0
mass        : 149.21
diameter    : 1.0
type        : M
typeid      : 0
body        : 4294967295
orientation : (1.0, 0.0, 0.0, 0.0)
mom. inertia: (0.0, 0.0, 0.0)
angular_momentum: (0.0, 0.0, 0.0, 0.0)
net_force   : (-0.03577616824239636, 0.00011565795558827875, -0.00029757467840749586)
net_energy  : 1.6000451769505004e-05
net_torque  : (0.0, 0.0, 0.0)
net_virial  : (0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

tag         : 1
position    : (-85.5386724327615, -0.0011469311065213559, 0.003397676785684532)
image       : (0, 0, 0)
velocity    : (0.004297220465239244, 0.0007165563681043077, 0.00045553398984320083)
acceleration: (-0.08434907347889421, 0.05107400362366826, 0.008670896149798588)
charge      : 0.

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))